# Purpose of this notebook
This Jupyter is used to split the data into train, validation and test. It is also used to generate the tokenizers for sequence and BOW tfidf.

**The notebook must be in the same folder as the results of top5 and quartiles notebook**

In [ ]:
from numpy import array
from collections import namedtuple

In [ ]:
import tensorflow as tf 
# test GPY tensor flow
from tensorflow.python.client import device_lib
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))# --> true
# CUDA with tensorflow support GPU? --> true
print(tf.test.is_built_with_cuda()) 
# GPU active? --> GPU and CPU
print(device_lib.list_local_devices())
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())# Keras using GPUS?

In [ ]:
import numpy as np
import collections 
import csv
import os
import pandas as pd
import pickle
import random
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import StratifiedShuffleSplit
from random import shuffle

In [ ]:
Representation = namedtuple('Representation', 'b0 b1 b1_int b1_iden b1_int_iden') # Tokenizers
#X
DataXModel = namedtuple('DataXModel', 'b0 b1 b1_int b1_iden b1_int_iden') # DataFrames single columns
#Y 
DataYModel = namedtuple('DataYModel', 'b0 b1 b1_int b1_iden b1_int_iden')# DataFrame two columns (Binary and categorical)

#Index
IndexTrain = namedtuple('Index', 'b0 b1 b1_int b1_iden b1_int_iden') # DataFrame

# This aproach can be binary and multi 
approach = 'multi'

In [ ]:
representations = ['b0','b1','b1_int','b1_iden','b1_int_iden']

In [ ]:
def description_vocab(dataset,representations,approach):
    
    print("Dataset:", dataset)
    
    multiple_list = collections.defaultdict(list) 
    df_index = pd.DataFrame(columns= ['ID','Type'])
    
    for rep in representations:
        #read data
        temp = pd.read_csv(f'{approach}_{rep}_{dataset}.csv') 
        
        
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=0)
        print("First split train /rest")
        column_name = ""

        if approach == 'binary':
            column_name = 'VULN_N' 
        else:
            column_name = 'TYPE_N' 

        X = temp['0'].to_frame()
        y = temp[column_name].to_frame()

        print("Original Shape:",temp.shape)
        sss.get_n_splits(X, y)  

        for train_index, test_val_index in sss.split(X, y):
            print("Train Shape:",train_index.shape)
            X_train, X = X.loc[train_index].copy(), X.loc[test_val_index].copy()
            y_train, y = y.loc[train_index].copy(), y.loc[test_val_index].copy()
            #add index to save
            df = pd.DataFrame({'ID':train_index,'Type':np.full(len(train_index),0)})
            df_index.append(df, ignore_index= True)        
        
        
        #SAVE x list    
        multiple_list['list_x_train'].append(X_train)
        multiple_list['list_y_train'].append(y_train)
        X = X.reset_index(drop= True).copy()
        y = y.reset_index().copy()
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
        sss.get_n_splits(X, y[column_name])  

        
        for val_index, test_index in sss.split(X, y[column_name]):
            print("val-test Shape:",val_index.shape,"-", test_index.shape)
            X_val, X_test = X.loc[val_index].copy(), X.loc[test_index].copy()
            y_val, y_test = y.loc[val_index].copy(), y.loc[test_index].copy()            
            #add index to save val 
            df = pd.DataFrame({'ID':y_val['index'],'Type':np.full(len(val_index),1)})
            df_index.append(df, ignore_index=True)    

            #add index to save test 
            df = pd.DataFrame({'ID':y_test['index'],'Type':np.full(len(test_index),2)})
            df_index.append(df, ignore_index=True)

            y_val, y_test = y_val[column_name].to_frame(),  y_test[column_name].to_frame()
        
        # Save Validation and test
        multiple_list['list_x_val'].append(X_val)
        multiple_list['list_y_val'].append(y_val)
        multiple_list['list_x_test'].append(X_test)
        multiple_list['list_y_test'].append(y_test)
        multiple_list['list_index'].append(df_index)                
                
        print("Representation:-tokenizer",rep)
        
        #create a Tokenizer object that ignore spaces and not punctuations
            token_obj = Tokenizer(filters=' ', lower= False,oov_token='OOVTOKEN')
        token_obj.fit_on_texts(X_train['0'])
        multiple_list['list_rep'].append(token_obj)
        
        print('length vocab:', len(token_obj.word_index))
        print('length Docs (# functions):', token_obj.document_count)
                
                      
        print(" ")
        print(" ")
    
    #named tuples
    Reps = Representation(*multiple_list['list_rep'])
    X_train = DataXModel(*multiple_list['list_x_train'])
    X_val = DataXModel(*multiple_list['list_x_val'])    
    X_test = DataXModel(*multiple_list['list_x_test'])
    Y_train = DataYModel(*multiple_list['list_y_train'])
    Y_val = DataYModel(*multiple_list['list_y_val'])
    Y_test = DataYModel(*multiple_list['list_y_test'])
    Index_train = IndexTrain(*multiple_list['list_index'])
    
        
    return (Reps,X_train, X_val, X_test, Y_train, Y_val, Y_test, Index_train)

In [ ]:
try:
    os.mkdir("BOW")
except OSError:
    print ("Creation of the directory /BOW failed")
try:
    os.mkdir("Tokenizer")
except OSError:
    print ("Creation of the directory \Tokenizer failed")
try:
    os.mkdir("Data_train_test")
except OSError:
    print ("Creation of the directory \Data_train_test failed")
try:
    os.mkdir("Index")
except OSError:
    print ("Creation of the directory \Index failed")

# Vocab description: RUSSELL

In [ ]:
tokenizer_Russel, x_train_Russel, x_val_Russel, x_test_Russel, y_train_Russel, y_val_Russel, y_test_Russel, index_Russel = description_vocab('Russell',representations,approach)

In [ ]:
y_train_Russel.b0.TYPE_N.unique()

# Vocab description: OUR

In [ ]:
tokenizer_OUR, x_train_OUR, x_val_OUR, x_test_OUR, y_train_OUR, y_val_OUR, y_test_OUR, index_OUR = description_vocab('OUR',representations,approach)

In [ ]:
y_train_OUR.b0.TYPE_N.unique()

# Vocab description: Juliet

In [ ]:
tokenizer_juliet, x_train_juliet, x_val_juliet, x_test_juliet, y_train_juliet, y_val_juliet, y_test_juliet, index_juliet = description_vocab('Juliet',representations,approach)

In [ ]:
y_train_juliet.b0.TYPE_N.unique()

In [ ]:
len(tokenizer_juliet.b0.word_index)

# SAVE partitions and tokenizers

In [ ]:
def save_tokenizers(named_tupled,dataset,approach):    
    for name, tokenizer in named_tupled._asdict().items():
        name = f'Tokenizer/{approach}_{dataset}_{name}.pickle'
        with open(name, 'wb') as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
def save_data_train(named_tuple,dataset,data_type,approach):
    for name, data in named_tuple._asdict().items():
        name = f'Data_train_test/{approach}_{data_type}_{dataset}_{name}.csv'
        data.to_csv(name,index=False)
        
def save_index(dataset,named_tuple,approach):
    
    for name, data in named_tuple._asdict().items():
        name = f'Index/{approach}_{dataset}_{name}.csv'
        data.to_csv(name,index=False)

In [ ]:
#Save named tuples of tokenizers by dataset and representation
save_tokenizers(tokenizer_juliet,'juliet',approach)
save_tokenizers(tokenizer_Russel,'Russell',approach)
save_tokenizers(tokenizer_OUR,'OUR',approach)

In [ ]:
#JULIET
save_data_train(x_train_juliet,'Juliet','Xtrain',approach)
save_data_train(x_val_juliet,'Juliet','Xval',approach)
save_data_train(x_test_juliet,'Juliet','Xtest',approach)
save_data_train(y_train_juliet,'Juliet','Ytrain',approach)
save_data_train(y_val_juliet,'Juliet','Yval',approach)
save_data_train(y_test_juliet,'Juliet','Ytest',approach)
save_index('Juliet',index_juliet,approach)

#RUSSELL
save_data_train(x_train_Russel,'Russell','Xtrain',approach)
save_data_train(x_val_Russel,'Russell','Xval',approach)
save_data_train(x_test_Russel,'Russell','Xtest',approach)
save_data_train(y_train_Russel,'Russell','Ytrain',approach)
save_data_train(y_val_Russel,'Russell','Yval',approach)
save_data_train(y_test_Russel,'Russell','Ytest',approach)
save_index('Russell',index_Russel,approach)

#OUR
save_data_train(x_train_OUR,'OUR','Xtrain',approach)
save_data_train(x_val_OUR,'OUR','Xval',approach)
save_data_train(x_test_OUR,'OUR','Xtest',approach)
save_data_train(y_train_OUR,'OUR','Ytrain',approach)
save_data_train(y_val_OUR,'OUR','Yval',approach)
save_data_train(y_test_OUR,'OUR','Ytest',approach)
save_index('OUR',index_OUR,approach)
